In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder

In [8]:
train_data = pd.read_csv('../data/train.csv')
test_data = pd.read_csv('../data/test.csv')
print(train_data[::15])

     PassengerId  Survived  Pclass  \
0              1         0       3   
15            16         1       2   
30            31         0       1   
45            46         0       3   
60            61         0       3   
75            76         0       3   
90            91         0       3   
105          106         0       3   
120          121         0       2   
135          136         0       2   
150          151         0       2   
165          166         1       3   
180          181         0       3   
195          196         1       1   
210          211         0       3   
225          226         0       3   
240          241         0       3   
255          256         1       3   
270          271         0       1   
285          286         0       3   
300          301         1       3   
315          316         1       3   
330          331         1       3   
345          346         1       2   
360          361         0       3   
375         